In [37]:
names= [name.strip() for name in open("names.txt")]

In [38]:
names[0:3]

['emma', 'olivia', 'ava']

In [39]:
chars=sorted(list(set("".join(names))))
stoi= {chars[i]: i+1 for i in range(len(chars)) }
stoi['.']=0

In [40]:
import torch
xs=[] # input
ys=[] # expected output
# prepare the dataset
#Network takes first letter in bigram as input and would predict the second letter as output
for name in names:
  char=["."]+list(name)+["."]
  for ch1, ch2 in zip(char, char[1:]):
    ix1=stoi[ch1]
    ix2=stoi[ch2]
    xs.append(ix1)
    ys.append(ix2)
xs=torch.tensor(xs)
ys=torch.tensor(ys)

In [41]:
import torch.nn.functional as F
num_samples=len(xs)
g=torch.Generator().manual_seed(2147483647)
w= torch.randn((27,27), generator=g, requires_grad=True)
xenc=F.one_hot(xs,num_classes=27).float()

In [42]:
for i in range(200):

  #Forward pass
  logits=xenc @ w
  ex=torch.exp(logits)
  softmax= ex/ex.sum(dim=1, keepdim=True)
  y_pred=ys[0]
  loss = -torch.log(softmax[range(num_samples), ys]).mean()
  # backward pass
  w.grad=None
  loss.backward()
  # Gradient descent
  lr=50
  w.data+=-lr*w.grad


In [43]:
print(loss.item())

2.462393045425415
